In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from collections import defaultdict
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score,classification_report

## Problem 1
#### Building in house evaluaion metrics for our models such as Fscore, Precesion and Recall


In [2]:
## Calculates the precision of the predicted labels
def get_precision(y_pred, y_test1, debug = False):
    # deal with npdarray
    y_pred = list(y_pred)
    y_true = list(y_test1)
    #---------
    n = len(y_pred);
    y_pred = [0 if v is None else v for v in y_pred]# deal with None type
    y_true = [0 if v is None else v for v in y_test1]# deal with None type
    true_positive = sum(y_pred[i]* y_test1[i] for i in range(n))
    if (0 == sum(y_pred)): return 0
    return true_positive*1.0/sum(y_pred)
    
## Calculates the recall of the predicted labels
def get_recall(y_pred, y_test1):
    # deal with npdarray
    y_pred = list(y_pred)
    y_true = list(y_test1)
    #---------
    n = len(y_pred);
    y_pred = list(map(int,[1 == l for l in y_test1]))# deal with None type
    y_true = list(map(int,[1 == l for l in y_test1]))# deal with None type
    true_positive = sum(y_pred[i]*y_test1[i] for i in range(n))
    if 0 == sum(y_test): return 0
    return true_positive*1.0/sum(y_test1)
    

## Calculates the f-score of the predicted labels
def get_fscore(y_pred, y_test1):
    precision = get_precision(y_pred, y_test1);
    if (0 == precision): return 0
    recall= get_recall(y_pred, y_test1);
    if (0 == recall): return 0
    beta = 1.0;
    # print("get_fscore:",(beta**2*precision+recall))
    fscore = (beta**2+1)*precision*recall/(beta**2*precision+recall);
    return fscore

### Downloading feat files, transform and observe them

In [3]:
#import requests

#filename = 'aclImdb_v1.tar.gz'
#url = u'http://ai.stanford.edu/~amaas/data/sentiment/' + filename
#r = requests.get(url)
#with open(filename, 'wb') as f:
    #f.write(r.content)

# ...extract zip file
#import tarfile

#tar = tarfile.open(filename, mode='r')
#tar.extractall()
#tar.close()

In [4]:
#Load Files and divide them features and labels
import sklearn
from sklearn.datasets import load_svmlight_file
X_test, y_test = load_svmlight_file("Test_labeledBow.feat")
X_train, y_train = load_svmlight_file("Train_labeledBow.feat")
y_train

array([ 9.,  7.,  9., ...,  4.,  2.,  2.])

In [5]:
#Labels are given in review score turning them binary by given threshold
y_test= np.where(y_test <= 4, 0, 1)
y_train=np.where(y_train <= 4, 0, 1)
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [6]:
#changing dtype for efficient memory usage 
import scipy.sparse._sparsetools
import scipy
X_test=scipy.sparse.lil_matrix(X_test, dtype='uint16')
X_train=scipy.sparse.lil_matrix(X_train, dtype='uint16')

In [7]:
#Changing to array as our test and train data in sparse matrix 
X_test_array=X_test.toarray()
X_train_array=X_train.toarray()

In [8]:
#Observing train data in dataframe
train_df=pd.DataFrame(X_train_array)
train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89517,89518,89519,89520,89521,89522,89523,89524,89525,89526
0,9,1,4,4,6,4,2,2,4,0,...,0,0,0,0,0,0,0,0,0,0
1,7,4,2,2,0,4,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
2,4,4,4,7,2,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,10,2,2,0,3,2,4,2,0,1,...,0,0,0,0,0,0,0,0,0,0
4,13,9,6,4,2,5,10,6,0,2,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Observing test data in dataframe
test_df=pd.DataFrame(X_test_array)
test_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89513,89514,89515,89516,89517,89518,89519,89520,89521,89522
0,7,4,2,5,5,1,3,1,6,3,...,0,0,0,0,0,0,0,0,0,0
1,9,4,4,0,3,1,10,1,12,1,...,0,0,0,0,0,0,0,0,0,0
2,21,7,4,5,7,8,0,5,8,4,...,0,0,0,0,0,0,0,0,0,0
3,13,5,8,8,4,2,6,4,8,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1,2,1,3,2,3,2,4,4,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# As you can see test data is less in columns thus we add default columns so that our test data will be suitable for model impl.
test_df['89523']='0'
test_df['89524']='0'
test_df['89525']='0'
test_df['89526']='0'
test_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89517,89518,89519,89520,89521,89522,89523,89524,89525,89526
0,7,4,2,5,5,1,3,1,6,3,...,0,0,0,0,0,0,0,0,0,0
1,9,4,4,0,3,1,10,1,12,1,...,0,0,0,0,0,0,0,0,0,0
2,21,7,4,5,7,8,0,5,8,4,...,0,0,0,0,0,0,0,0,0,0
3,13,5,8,8,4,2,6,4,8,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1,2,1,3,2,3,2,4,4,...,0,0,0,0,0,0,0,0,0,0


## Problem 2 
### Majority Baseline


In [11]:
#in this question we will use Majority Baseline Model. 
#As data is even between neg and pos reviews we assign majority reviews as pos
#The label for the test data would be the majority class found in training data that is positive as we assigned
y_pred = [1]*len(test_df)
#Evaluating model performance
print('Fscore :', get_fscore(y_pred, y_test))
print('Recall', get_recall(y_pred, y_test))
print('Precesion',get_precision(y_pred, y_test, debug = False))


Fscore : 0.666666666667
Recall 1.0
Precesion 0.5


## Problem 3
### Length Baseline

#### Choosing threshold that maximizes accuracy

Here we are trying to pridict the optimal threshold so that we can get higher level of accuracy. The below function will produce the optimal range that maximizes fscore. In other words, we are assuming that the length of the review may also determine whether the review is positive or not. Therefore, rather than blindly choosing the optimal range of length of the review, we build the following function that we use later to set our threshold and also will check how far it is optimal.

In [14]:
def review_length_threshold(df, label, plot_flag = True):
    ## YOUR CODE HERE
    # print("load training file")
    #words,labels = load_file(training_file, True)
    Dict_len = defaultdict(set)
    for w in df: Dict_len[w].add(w)
    # Evaluation depending on fscore
    Min = min(Dict_len)
    Max = max(Dict_len) 
    ThreRange=  range(Min,Max+1)
    FscoreL = dict([(f1_score(list(map(int,[w > Thres for w in df])), label, average="macro"),Thres) for Thres in ThreRange])
    Thres_opt = FscoreL[max(FscoreL)]
    if plot_flag:
         print("Range of thresholds:",Min,"to",Max, " with optimal threshold:", Thres_opt+1)
    del df
    del label
    return ThreRange, Thres_opt

In [15]:
train_df_length=train_df.sum(axis=1)
train_df_length.head(5)

0    138
1    114
2    115
3    104
4    232
dtype: int64

In [16]:
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [17]:
review_length_threshold(train_df_length, y_train, plot_flag = True)

Range of thresholds: 10 to 2476  with optimal threshold: 194


/opt/anaconda/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(range(10, 2477), 193)

###### As we can see the function produced us optimal threshold of 194, which means the text less than these threshold is considered as negative while above it is considered as positive

#### Length for train

In [18]:
#Adding up all columns for each row to get length of text for each review
X_train_array_length=X_train_array.sum(axis=1)
X_train_array_length

array([138, 114, 115, ..., 231, 153, 138], dtype=uint64)

In [19]:
data_train1=np.where(X_train_array_length < 194, 0, 1)

In [20]:
data_train1

array([0, 0, 0, ..., 1, 0, 0])

In [21]:
print('Fscore :', get_fscore(data_train1, y_train))
print('Recall', get_recall(data_train1, y_train))
print('Precesion',get_precision(data_train1, y_train, debug = False))

Fscore : 0.671514852706
Recall 1.0
Precesion 0.505474113936


###### Case two: pos < 194, neg> 194

In [22]:
#case two, Now lets imagine that all reviews less than 194 are positive and the ones more than 194 is negative
# we assume that people write more when they did not like the moview. let us check it
data_train2=np.where(X_train_array_length < 194, 1, 0)

In [ ]:
data_train2

In [23]:
print('Fscore :', get_fscore(data_train2, y_train))
print('Recall', get_recall(data_train2, y_train))
print('Precesion',get_precision(data_train2, y_train, debug = False))

Fscore : 0.662968882204
Recall 1.0
Precesion 0.49585149768


###### Case 3: choosing randomly any value, pos>300 neg< 300

In [40]:
data_train3=np.where(X_train_array_length >300, 0, 1)
data_train3

array([1, 1, 1, ..., 1, 1, 1])

In [41]:
print('Fscore :', get_fscore(data_train3, y_train))
print('Recall', get_recall(data_train3, y_train))
print('Precesion',get_precision(data_train3, y_train, debug = False))

Fscore : 0.662437073305
Recall 1.0
Precesion 0.49525675397


#### Length for test

In [42]:
X_test_array_length=X_test_array.sum(axis=1)
X_test_array_length

array([152, 141, 302, ..., 258, 167, 142], dtype=uint64)

In [43]:
data_test1=np.where(X_test_array_length < 194, 0, 1)
data_test1

array([0, 0, 1, ..., 1, 0, 0])

In [44]:
print('Fscore :', get_fscore(data_test1, y_test))
print('Recall', get_recall(data_test1, y_test))
print('Precesion',get_precision(data_test1, y_test, debug = False))

Fscore : 0.659937888199
Recall 1.0
Precesion 0.492468134415


###### Case two: pos < 194, neg> 194

In [46]:
data_test2=np.where(X_test_array_length < 194, 1, 0)
data_test2

array([1, 1, 0, ..., 0, 1, 1])

In [48]:
print('Fscore :', get_fscore(data_test2, y_test))
print('Recall', get_recall(data_test2, y_test))
print('Precesion',get_precision(data_test2, y_test, debug = False))

Fscore : 0.67138450372
Recall 1.0
Precesion 0.505326413548


###### Case 3: choosing randomly any value, pos>300 neg< 300

In [49]:
data_test3=np.where(X_test_array_length  >300, 1, 0)
data_test3

array([0, 0, 1, ..., 0, 0, 0])

In [51]:
print('Fscore :', get_fscore(data_test3, y_test))
print('Recall', get_recall(data_test3, y_test))
print('Precesion',get_precision(data_test3, y_test, debug = False))

Fscore : 0.665219057351
Recall 1.0
Precesion 0.498373205742


###### As you can see, all scores are very similar to each other, which means that disribution of data is very normal.

## Problem 4
### Naive Bayes Classifier

In [11]:
#initialize NB classifier
print("Naive-Bayes Classifier")
nb = GaussianNB()
#fit your classifier to training set
nb = nb.fit(train_df, y_train)


Naive-Bayes Classifier


In [ ]:
#As you saw above I was training my model with full data, but for prediction I use less data as my memory is not handling it
y_pred1 = nb.predict(test_df[:15000])


In [24]:
print('Fscore :', get_fscore(y_pred1, y_test[:15000]))
print('Recall', get_recall(y_pred1, y_test[:15000]))
print('Precesion',get_precision(y_pred1, y_test[:15000], debug = False))

Fscore : 0.915222562012
Recall 1.0
Precesion 0.843696162882


## Problem 5
### Decision Tree Classifier


In [11]:
#Decision Tree Classifier
print("Decision Tree Classifier")
dt = tree.DecisionTreeClassifier()
dt = dt.fit(train_df, y_train)

Decision Tree Classifier


In [14]:
#As you saw above I was training my model with full data, but for prediction I use less data as my memory is not handling it
y_pred = dt.predict(test_df[:15000])


In [15]:
print('Fscore :', get_fscore(y_pred, y_test[:15000]))
print('Recall', get_recall(y_pred, y_test[:15000]))
print('Precesion',get_precision(y_pred, y_test[:15000], debug = False))


Fscore : 1.0
Recall 1.0
Precesion 1.0


### Problem 6

As we can observe from all 5 models, the most accurate ones were Naive BAyes and Decision Tree. The other two self made assumptions unfortunately did not give good results if we look at our model evaluation metrics. 

## Problem 7

In [6]:
pip install --user mxnet

  Using cached https://files.pythonhosted.org/packages/a4/3e/07f6d61d8e0a5d50c6f128791a617d7b222224c52dacba04855a0b6ecfe6/mxnet-1.5.0-py2.py3-none-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gluonnlp

  Stored in directory: C:\Users\Ibragim\AppData\Local\pip\Cache\wheels\3e\e7\3e\9cdf8ad7fce112fde2f4a52604045e5dd80f84d645bedb70c7
Successfully built gluonnlp
Note: you may need to restart the kernel to use updated packages.


In [2]:
 import warnings
warnings.filterwarnings('ignore')

from mxnet import gluon
from mxnet import nd
import gluonnlp as nlp
import re

In [3]:
fasttext_simple = nlp.embedding.create('fasttext', source='wiki.simple')

In [4]:
nlp.embedding.list_sources('glove')
#We can  create vocabulary by using vocabulary of pre-trained word embeddings, such as GloVe. 
#Below are a few pre-trained file names under the GloVe word embedding.


['glove.42B.300d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d',
 'glove.6B.50d',
 'glove.840B.300d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d']

In [26]:
#we use a smaller word embedding file, such as the 300-dimensional one
glove_6b300d = nlp.embedding.create('glove', source='glove.6B.300d')

Embedding file glove.6B.300d.npz is not found. Downloading from Gluon Repository. This may take some time.


##### Dependencies 

In [28]:
#we create vocabulary by using all the tokens from  pre trained glove:glove_6b50d
vocab = nlp.Vocab(nlp.data.Counter(glove_6b300d.idx_to_token))
vocab.set_embedding(glove_6b300d)

In [8]:
#we need to define cosine similarity
from mxnet import nd
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))

In [9]:
#Word Similarity
def norm_vecs_by_row(x):
    return x / nd.sqrt(nd.sum(x * x, axis=1) + 1E-10).reshape((-1,1))

def get_knn(vocab, k, word):
    word_vec = vocab.embedding[word].reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(vocab.embedding.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_vec)
    indices = nd.topk(dot_prod.reshape((len(vocab), )), k=k+1, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    # Remove unknown and input tokens.
    return vocab.to_tokens(indices[1:])

In [10]:
#We can  apply pre-trained word embeddings to the word analogy test
def get_top_k_by_analogy(vocab, k, word1, word2, word3):
    word_vecs = vocab.embedding[word1, word2, word3]
    word_diff = (word_vecs[1] - word_vecs[0] + word_vecs[2]).reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(vocab.embedding.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_diff)
    indices = nd.topk(dot_prod.reshape((len(vocab), )), k=k, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    return vocab.to_tokens(indices)

In [11]:
# we are measuring here accuracy of analogy cosine similarity 
#between vec(‘word1’)+vec(‘word2’)-vec(‘word3’) and vec(‘word4’).  
def cos_sim_word_analogy(vocab, word1, word2, word3, word4):
    words = [word1, word2, word3, word4]
    vecs = vocab.embedding[words]
    return cos_sim(vecs[1] - vecs[0] + vecs[2], vecs[3])

##### Analogy Test 1 with pre trained  glove_6b300d

###### Semantic


In [73]:
# Question: mother, father, sister brother
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'mother', 'father', 'sister')) #right analogy
# Question: Europe euro USA dollar 
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'Europe', 'euro', 'USA')) #wrong prediction
# Question: Detroit Michigan Sacramento California
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'Detroit', 'Michigan', 'Sacramento ')) #wrong prediction
# Question: beijing, china, tokyo, japan
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'beijing', 'china', 'tokyo')) #wrong prediction

Top Analogy: ['brother']
Top Analogy: ['euro']
Top Analogy: ['<unk>']
Top Analogy: ['tokyo']


In [74]:
print('Cosine sim:', cos_sim_word_analogy(vocab,  'mother', 'father', 'sister', 'brother'))
print('Cosine sim:', cos_sim_word_analogy(vocab,  'Europe', 'euro', 'USA', 'euro'))
print('Cosine sim:', cos_sim_word_analogy(vocab,  'Detroit', 'Michigan', 'Sacramento', 'California'))
print('Cosine sim:', cos_sim_word_analogy(vocab,  'beijing', 'china', 'tokyo', 'tokyo'))

Cosine sim: 
[0.80230254]
<NDArray 1 @cpu(0)>
Cosine sim: 
[1.0000001]
<NDArray 1 @cpu(0)>
Cosine sim: 
[nan]
<NDArray 1 @cpu(0)>
Cosine sim: 
[0.8107581]
<NDArray 1 @cpu(0)>


#### Syntactic 

In [61]:
# Question:amazing amazingly rapid rapidly
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'amazing', 'amazingly', ' rapid')) #wrong prediction
# Question: logical illogical honest dishonest
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'logical', 'illogical', 'honest')) #wrong prediction
# Question: 'bad', 'worst', 'big', 'biggest'
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'bad', 'worst', 'big')) #wrong prediction
# Question:'do', 'did', 'go', 'went'
print( 'Top Analogy:',get_top_k_by_analogy(vocab, 1, 'do', 'did', 'go')) #right Prediction

Top Analogy: ['amazingly']
Top Analogy: ['honest']
Top Analogy: ['worst']
Top Analogy: ['went']


In [63]:
print('Cosine sim:', cos_sim_word_analogy(vocab,  'amazing', 'amazingly', ' rapid', 'amazingly'))
print('Cosine sim:', cos_sim_word_analogy(vocab,  'logical', 'illogical', 'honest', 'honest'))
print('Cosine sim:', cos_sim_word_analogy(vocab,  'bad', 'worst', 'big', 'worst'))
print('Cosine sim:', cos_sim_word_analogy(vocab,  'do', 'did', 'go', 'went'))

Cosine sim: 
[0.4422674]
<NDArray 1 @cpu(0)>
Cosine sim: 
[0.6742419]
<NDArray 1 @cpu(0)>
Cosine sim: 
[0.6712073]
<NDArray 1 @cpu(0)>
Cosine sim: 
[0.77813184]
<NDArray 1 @cpu(0)>


##### Analogy Test 2 with pre trained  glove_6b300d

###### Semantic

In [5]:
glove_6B50d = nlp.embedding.create('glove', source='glove.6B.50d')

In [6]:
vocab2 = nlp.Vocab(nlp.data.Counter(glove_6B50d.idx_to_token))
vocab2.set_embedding(glove_6B50d)

In [12]:
# Question: mother, father, sister brother
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'mother', 'father', 'sister')) #right analogy


Top Analogy: ['brother']


In [13]:
# Question: Europe euro USA dollar 
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'Europe', 'euro', 'USA')) #wrong prediction


Top Analogy: ['euro']


In [14]:
# Question: Detroit Michigan Sacramento California
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'Detroit', 'Michigan', 'Sacramento ')) #wrong prediction


Top Analogy: ['<unk>']


In [15]:
# Question: beijing, china, tokyo, japan
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'beijing', 'china', 'tokyo')) #right prediction

Top Analogy: ['japan']


###### Syntactic

In [16]:
# Question:amazing amazingly rapid rapidly
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'amazing', 'amazingly', ' rapid')) #wrong prediction


Top Analogy: ['apppointed']


In [17]:
# Question: logical illogical honest dishonest
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'logical', 'illogical', 'honest')) #wrong prediction


Top Analogy: ['irresponsible']


In [18]:
# Question: 'bad', 'worst', 'big', 'biggest'
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'bad', 'worst', 'big')) #right prediction


Top Analogy: ['biggest']


In [19]:
# Question:'do', 'did', 'go', 'went'
print( 'Top Analogy:',get_top_k_by_analogy(vocab2, 1, 'do', 'did', 'go')) #right Prediction

Top Analogy: ['went']


##### Of course, due to limitations of my laptop memory capacity, I was not able to load larger pre trained embeddings like Google News, though I was able to really understand how word embeddings are  formed. It is interesting to note if I compare two results, higher dimensional embedding was less accurate then less dimensional. Also, I guess the size of vocabulary would play significant role as I can say that , may be because of limited wocabulary, two of the completed anology tests did  gave me expected result of at least 75% accuracy. 

# Problem 8

Word embedding is a distributed representation of words in a vector space. It involves a mathematical embedding from a space
with one dimension per word to a continuous vector space with much lower dimension. Word embeddings greatly enhanced word representation. Vectorised presentation of words, however often fail to correctly recognize antonymous words like hot and cold, because of context similarity.  The main reason for this would be the fact that pre-trained word embeddings are trained from large amount of corpus where word distributions share similar contexts. This may provoke case that it would be difficult to discriminate antonyms from synonyms. Therefore, this problem should be resolved and now  a few papers are published to address it as tasks like semantic textual similarity will bring more accurate results if antonym-sensitive embeddings are created. 

[Dou, Z., Wei, W., & Wan, X. (2018). Improving Word Embeddings for Antonym Detection Using Thesauri and SentiWordNet. Natural Language Processing and Chinese Computing Lecture Notes in Computer Science, 67–79. doi: 10.1007/978-3-319-99501-4_6]

In [20]:
get_knn(vocab, 10, 'enter')

['allow',
 'leave',
 'entering',
 'must',
 'able',
 'stay',
 'take',
 'to',
 'decide',
 'supposed']

In [21]:
get_knn(vocab, 10, 'increase') #wow there are a lot of mismatch

['increases',
 'increased',
 'increasing',
 'decrease',
 'reduction',
 'reducing',
 'reduced',
 'growth',
 'reduce',
 'higher']

### Problem 9 

##### First tripler

In [64]:
#Season to Month
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'july', 'summer', ' January')) #wrong prediction, winter is right

Top Analogy: ['summer']


In [72]:
#animal to its production
print( 'Top Analogy:', get_top_k_by_analogy(vocab,1, 'cow', 'milk', 'chicken')) #wrong prediction, egg is right

Top Analogy: ['chicken']


In [66]:
#food to category 
print( 'Top Analogy:', get_top_k_by_analogy(vocab, 1, 'potatoe', 'vegetable', ' apple')) #wrong prediction, fruit is right

Top Analogy: ['vegetable']


#### Second Tripler

In [20]:
#Season to Month
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'july', 'summer', ' January')) #wrong prediction, winter is right


Top Analogy: ['foodies']


In [22]:
#animal to its production
print( 'Top Analogy:', get_top_k_by_analogy(vocab2,1, 'cow', 'milk', 'chicken')) #wrong prediction, egg is right

Top Analogy: ['sauce']


In [23]:
#food to category 
print( 'Top Analogy:', get_top_k_by_analogy(vocab2, 1, 'potatoe', 'vegetable', ' apple')) #wrong prediction, fruit is right

Top Analogy: ['vegetable']


### Reference
[1] Yoshua Bengio, Ducharme Rejean &Vincent Pascal. A Neural Probabilistic Language Model. 2001. https://papers.nips.cc/paper/1839-a-neural-probabilistic-language-model.pdf

[2] Yoshua Bengio, Ducharme Rejean, Vincent Pascal & Janvin Christian. A Neural Probabilistic Language Model. March 2003. http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

[3] Collobert Ronan, & Weston Jason. A Unified Architecture for Natural Language Processing: Deep Neural Networks with Multitask Learning. 2008. https://ronan.collobert.com/pub/matos/2008_nlp_icml.pdf

[4] Tomas Mikolov, Greg Corrado, Kai Chen & Jeffrey Dean. Efficient Estimation of Word Representations in Vector Space. September 2013. https://arxiv.org/pdf/1301.3781.pdf

[5] Gluon-nlp.mxnet.io. (2019). Pre-trained Word Embeddings — gluonnlp 0.8.1 documentation. [online] Available at: https://gluon-nlp.mxnet.io/examples/word_embedding/word_embedding.html [Accessed 7 Oct. 2019].